In [ ]:
import evaluate
import datasets
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer

In [ ]:
model_id = 'kreimben/CodeMind-gemma'

tokenizer = AutoTokenizer.from_pretrained('kreimben/CodeMind-gemma')
tokenizer.padding_side = 'left'
model = AutoModelForCausalLM.from_pretrained('kreimben/CodeMind-gemma')
model = model.to('cuda')

In [ ]:
submission_dataset = datasets.load_dataset('kreimben/leetcode_user_submissions_only_python', split='train').to_pandas()
submission_dataset = submission_dataset[['title_slug', 'question_hints', 'question_content', 'content']]
captions_dataset = datasets.load_dataset('kreimben/leetcode_with_youtube_captions', split='train').to_pandas()[
    ['title_slug', 'question_hints', 'question_content', 'cc_content']]
captions_dataset.rename(columns={'cc_content': 'content'}, inplace=True)

dataset = pd.concat([submission_dataset, captions_dataset])

del submission_dataset, captions_dataset

# dataset.head(3)

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)

In [ ]:
GEMMA_2B_IT_MODEL_PREFIX_TEXT = """Below is an coding test problem. Solve the question."""


def generate_prompt(data_point):
    return f"""<start_of_turn>user {GEMMA_2B_IT_MODEL_PREFIX_TEXT}\n
I don't know {data_point['title_slug']} problem. give me the insight or appoach.\n
this is problem's hint.\n{data_point["question_hints"]}\n
here are some content of question.\n{data_point["question_content"]}<end_of_turn>
<start_of_turn>model {data_point["content"]}<end_of_turn>"""

In [ ]:
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)
dataset = dataset.shuffle(seed=42)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
dataset = dataset.remove_columns(['title_slug', 'question_hints', 'question_content', 'content', '__index_level_0__'])

In [ ]:
ds = dataset.train_test_split(test_size=.1)
test_dataset = ds['test']

In [ ]:
# Evaluate the fine-tuned model
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}


# Define the trainer
trainer = Trainer(
    model=model,
    # args=training_args,
    tokenizer=tokenizer,
)

In [ ]:
evaluation_metrics = trainer.evaluate(eval_dataset=dataset, metric_key_prefix="test")
print(f"Evaluation metrics: {evaluation_metrics}")